In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils import to_categorical
from tcn import TCN


# 读取数据集
df = pd.read_csv('fake_transactional_data_24.csv')


In [2]:
# 分类的目标变量
target = 'to_randomly_generated_account'

# 为分类任务，我们需要对目标变量进行独热编码
target_encoder = OneHotEncoder(sparse=False)
y = target_encoder.fit_transform(df[[target]])

/Users/liubohan/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
# 对于特征，我们需要对分类特征进行独热编码，对数值特征进行标准化
ct = ColumnTransformer([
    ("oh_enc", OneHotEncoder(sparse=False), ['from_totally_fake_account']),  # 假设这是分类特征
    ("scaler", StandardScaler(), ['monopoly_money_amount'])  # 假设这是数值特征
], remainder='drop')  # 其余特征我们不用

X = ct.fit_transform(df.drop(columns=[target]))

/Users/liubohan/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


: 

In [3]:
# 分割数据为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 构建TCN模型
model = Sequential([
    TCN(input_shape=(X_train.shape[1], 1),
        nb_filters=64,
        kernel_size=2,
        nb_stacks=1,
        dilations=[1, 2, 4, 8, 16, 32],
        padding='causal',
        use_skip_connections=True,
        dropout_rate=0.2,
        return_sequences=False,
        activation='relu',
        kernel_initializer='he_normal'),
    Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.002), loss='categorical_crossentropy', metrics=['accuracy'])

NameError: name 'X' is not defined

In [ ]:
# TCN需要三维输入，因此我们增加一个维度到我们的数据中
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# 训练模型
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

In [ ]:
# 评估模型
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

# 进行预测
predictions = model.predict(X_test)

# 将预测结果转换为标签
predicted_labels = target_encoder.inverse_transform(predictions)